In [22]:
import pandas as pd
import urllib.request
import json
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import numpy as np  
import folium
import re

In [23]:
# Load data
path = "D:\\OneDrive\\energia_lab\\data\\"
path1 = "D:\\OneDrive\\energia_lab\\streamlit\\data_steamlit\\"


df = pd.read_csv(path + 'df_unidos.csv', nrows=3)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    3 non-null      int64  
 1   DatGeracaoConjuntoDados       3 non-null      object 
 2   NomAgente                     3 non-null      object 
 3   IdeEmpresaProponenteProjeto   3 non-null      int64  
 4   DscCodProjeto                 3 non-null      object 
 5   DscTituloProjeto              3 non-null      object 
 6   DscTipologia                  3 non-null      object 
 7   VlrCustoTotal                 3 non-null      float64
 8   VlrRcbGlobal                  1 non-null      float64
 9   VlrEnergiaEconomizadaTotal    3 non-null      float64
 10  VlrRetiradaDemandaPontaTotal  3 non-null      float64
 11  DscObjetivo                   0 non-null      float64
 12  DscJustificativa              2 non-null      object 
 13  DatInicio

In [24]:
# Read the CSV file
date = pd.read_csv(path + 'df_unidos.csv', usecols=['DatInicioProjeto', 'DatConclusaoProjeto'], parse_dates=['DatInicioProjeto', 'DatConclusaoProjeto'])

In [25]:
projeto1 = date['DatInicioProjeto'].min()
df_projeto1 = pd.DataFrame([projeto1], columns=['DatInicioProjeto'])
df_projeto1.to_csv(path1+'projeto1.csv', index=False)

In [26]:
termino_ultimo_proj = date['DatConclusaoProjeto'].max()
df_termino_ultimo_proj = pd.DataFrame([termino_ultimo_proj], columns=['DatConclusaoProjeto'])
df_termino_ultimo_proj.to_csv(path1+'termino_ultimo_proj.csv', index=False)

In [27]:
# Calculate the difference in years
date['Duration_Years'] = (date['DatConclusaoProjeto'] - date['DatInicioProjeto']).dt.days / 365.25

# Calculate the mean of 'Duration_Years'
Duration_Years_projects_mean = date['Duration_Years'].mean()

# Convert the mean value into a DataFrame
df_Duration_Years_projects_mean = pd.DataFrame([Duration_Years_projects_mean], columns=['Duration_Years'])

# Save the DataFrame to a .csv file
df_Duration_Years_projects_mean.to_csv(path1+'Duration_Years_projects_mean.csv', index=False)

In [65]:

df_unidos = pd.read_csv(path + 'df_unidos.csv')


In [67]:
df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    72 non-null     int64  
 1   DatGeracaoConjuntoDados       72 non-null     object 
 2   NomAgente                     72 non-null     object 
 3   IdeEmpresaProponenteProjeto   72 non-null     int64  
 4   DscCodProjeto                 72 non-null     object 
 5   DscTituloProjeto              72 non-null     object 
 6   DscTipologia                  72 non-null     object 
 7   VlrCustoTotal                 72 non-null     float64
 8   VlrRcbGlobal                  37 non-null     float64
 9   VlrEnergiaEconomizadaTotal    72 non-null     float64
 10  VlrRetiradaDemandaPontaTotal  72 non-null     float64
 11  DscObjetivo                   7 non-null      object 
 12  DscJustificativa              34 non-null     object 
 13  DatInic